# 1. layer and block

In [1]:
import torch
from torch import nn

/opt/homebrew/Caskroom/miniforge/base/envs/Atari/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.2647,  0.1254,  0.0784, -0.0398, -0.1507, -0.0325, -0.2295,  0.0818,
         -0.2620, -0.3462],
        [ 0.1897,  0.1873,  0.2611, -0.1057, -0.2237,  0.1050, -0.2661,  0.0588,
         -0.2029, -0.2887]], grad_fn=<AddmmBackward0>)

## 1.1 自定义块

In [5]:
from torch.nn import functional as F

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [7]:
net = MLP()
net(X)

tensor([[-0.2363,  0.0557, -0.0316,  0.0048, -0.1877,  0.0591,  0.0633,  0.1861,
          0.1684,  0.0116],
        [-0.1220, -0.0128, -0.2427,  0.0213, -0.3506,  0.0814, -0.0737,  0.2226,
          0.0656, -0.1820]], grad_fn=<AddmmBackward0>)

## 1.2 Sequential

自定义的sequential，实现和sequential 相同的 功能

In [8]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
            
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)  

tensor([[-0.0239,  0.0338,  0.2586, -0.0771,  0.1172,  0.1354, -0.0220, -0.1866,
          0.1070,  0.1103],
        [-0.0342, -0.0453,  0.3676, -0.1687,  0.0667,  0.0856, -0.0581, -0.1386,
          0.1124,  0.1119]], grad_fn=<AddmmBackward0>)

## 1.3 在正向传播中执行代码

这个例子想要说明，我们可以通过继承module的方法做更自由的代码实现

尤其是forward的过程，我们可以添加很多自己想要的实现

举个例子

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(-0.4288, grad_fn=<SumBackward0>)

## 1.4 混合搭配各种组合module

感觉像在搭积木

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
        
        self.linear = nn.Linear(32, 10)
        
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(10, 20), FixedHiddenMLP())
chimera(X)
        

tensor(-0.1289, grad_fn=<SumBackward0>)

# 2. 参数管理

In [12]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.5018],
        [-0.4941]], grad_fn=<AddmmBackward0>)

## 2.1 参数管理

In [13]:
print(net[2].state_dict()) # 最后一层的参数

OrderedDict([('weight', tensor([[-0.2869, -0.3520, -0.0393,  0.0051, -0.1816,  0.1800, -0.2032, -0.1351]])), ('bias', tensor([-0.0825]))])


In [14]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0825], requires_grad=True)
tensor([-0.0825])


In [15]:
net[2].weight.grad == None # 拿到对应的梯度

True

In [16]:
net[0]

Linear(in_features=4, out_features=8, bias=True)

In [20]:
[(name, param.shape) for name, param in net[0].named_parameters()] # 拿到对应的参数名字以及形状

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]

In [21]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

In [22]:
net.state_dict()['2.bias'].data

tensor([-0.0825])

## 2.2 嵌套块收集参数

In [24]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4))

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1()) # 添加的时候，给个名字
        
    return net
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.5172],
        [-0.5170]], grad_fn=<AddmmBackward0>)

In [25]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

## 2.3 初始化

In [27]:
def init_normal(m):
    if type(m) == nn.Linear:
        # nn.init 中包含很多相关的初始化函数
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
        
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0038,  0.0145,  0.0095, -0.0103]), tensor(0.))

In [28]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [34]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0552,  0.0269, -0.5893,  0.1143])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


## 2.4 自定义初始化

In [38]:
def my_init(m):
    if type(m) == nn.Linear:
        print("init: ",
             *[(name, param.shape) for name, param in m.named_parameters()][0])
        
        nn.init.uniform_(m.weight, -10, 10)
        # 将绝对值 大于5的 取出来
        m.weight.data *= m.weight.data.abs() >= 5
        
net.apply(my_init)
net[0].weight

init:  weight torch.Size([8, 4])
init:  weight torch.Size([1, 8])


Parameter containing:
tensor([[ 6.7864,  0.0000,  9.0977, -9.7963],
        [-5.9737,  0.0000, -0.0000, -0.0000],
        [ 5.0594, -5.7662,  7.5889, -0.0000],
        [-0.0000,  5.4502, -5.9764, -6.8746],
        [-6.2652,  5.8229, -8.1528,  5.2202],
        [-8.0869,  0.0000,  5.8413,  7.0939],
        [-0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0000,  0.0000,  7.1868,  0.0000]], requires_grad=True)

In [39]:
# 更暴力的初始化方法，直接获取net中的weight，对其进行设置

net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, 10.0977, -8.7963])

## 2.5 参数绑定

In [40]:
shared = nn.Linear(8, 8)

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 4))

net(X)

tensor([[-0.0715, -0.1623, -0.0285,  0.3051],
        [-0.0740, -0.1606, -0.0318,  0.3066]], grad_fn=<AddmmBackward0>)

In [41]:
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


In [42]:
net[2].weight.data[0,0] = 100
net[4].weight.data[0, 0]

tensor(100.)

# 3. 自定义层

In [43]:
class CenterLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()
    
layer = CenterLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

## 3.1 将自定义层作为组建放到model中

In [44]:
# 没有任何区别，跟正常使用
net = nn.Sequential(nn.Linear(8, 16), CenterLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(2.3283e-09, grad_fn=<MeanBackward0>)

## 3.2 带参数的图层

In [45]:
class MyLinear(nn.Module):
    def __init__(self, in_units, out_units):
        super().__init__()
        # 所有加梯度加名字的地方
        self.weight = nn.Parameter(torch.randn(in_units, out_units))
        self.bias = nn.Parameter(torch.zeros(out_units))
        
    def forward(self, X):
        
        linear = torch.matmul(X , self.weight.data) + self.bias.data
        return F.relu(linear)

In [46]:
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-0.8259,  0.3679,  0.9901],
        [-1.0777, -0.9172,  0.7317],
        [-1.7821,  1.0496,  1.6881],
        [-1.3052, -0.4197, -0.9922],
        [ 0.0969, -0.3633,  0.7659]], requires_grad=True)

In [48]:
[(x, y.shape) for x, y in dense.named_parameters()]

[('weight', torch.Size([5, 3])), ('bias', torch.Size([3]))]

# 4. 读写文件

## 4.1 加载和保存单个张量

In [50]:
x = torch.arange(4)
torch.save(x, './models/x-file')

x2 = torch.load('./models/x-file')
x2

tensor([0, 1, 2, 3])

## 4.2 加载和保存list

In [51]:
y = torch.zeros(3)
torch.save([x, y], './models/list-file')

x2, y2 = torch.load('./models/list-file')

In [52]:
x2, y2

(tensor([0, 1, 2, 3]), tensor([0., 0., 0.]))

## 4.3 加载和保存字典

In [53]:
mydict = {'x':x, 'y': y}

In [54]:
torch.save(mydict, './models/dict_file')
mydict2 = torch.load('./models/dict_file')
print(mydict2)

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0.])}


## 4.4 加载和保存模型参数

In [55]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))
    
net = MLP()
X = torch.randn(size=(10, 20))
Y = net(X)

In [58]:
Y.shape

torch.Size([10, 10])

In [61]:
torch.save(net.state_dict(), 'models/mlp_param')

In [62]:
clone = MLP()
clone.load_state_dict(torch.load('models/mlp_param'))

<All keys matched successfully>

In [63]:
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [64]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])